In [1]:
import sys
# sys.path.append('/Users/joachim/texjs/lva/IntroSC/ASC-ODE/build/mechsystem')
sys.path.append('../build/mechsystem')

from mass_spring import *

from pythreejs import *

In [2]:
import math

mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

def add_spring(c1, c2, k):
    p1 = mss[c1].pos
    p2 = mss[c2].pos
    L = math.dist(p1, p2)
    mss.add(Spring(L, k, (c1, c2)))

k_support = 40000   
k_spring  = 8000    

# L-shaped crane
f1 = mss.add(Fix((0.0, 0.0, 0.0)))   # bottom
f2 = mss.add(Fix((0.0, 0.0, 2.0)))   # vertical up
f3 = mss.add(Fix((2.0, 0.0, 2.0)))   # horizontal right (90°)

# "beams" as fixes ??
add_spring(f1, f2, k_support)
add_spring(f2, f3, k_support)

# one mass at the end
m = mss.add(Mass(2.0, (2.0, 0.6, 1.2)))   # slightly offset in y gives vibration? 
add_spring(f3, m, k_spring)




In [3]:
from pythreejs import *
from time import sleep

masses = [
    Mesh(SphereBufferGeometry(0.2, 16, 16),
         MeshStandardMaterial(color='red'),
         position=(mm.pos[0], mm.pos[1], mm.pos[2]))
    for mm in mss.masses
]

fixes = [
    Mesh(SphereBufferGeometry(0.25, 16, 16),
         MeshStandardMaterial(color='blue'),
         position=(ff.pos[0], ff.pos[1], ff.pos[2]))
    for ff in mss.fixes
]

def get_segments():
    segs = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        segs.append([(pA[0],pA[1],pA[2]), (pB[0],pB[1],pB[2])])
    return segs

springgeo = LineSegmentsGeometry(positions=get_segments())
springmat = LineMaterial(linewidth=3, color='cyan')
springs = LineSegments2(springgeo, springmat)

axes = AxesHelper(1)
view_width, view_height = 600, 400
camera = PerspectiveCamera(position=[8, 6, 6], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)
renderer


Renderer(camera=PerspectiveCamera(aspect=1.5, position=(8.0, 6.0, 6.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0, …

In [4]:
# from time import sleep
# from pythreejs import *

# for i in range(3000):
#     mss.simulate(0.005, 20)

#     for mm, vis in zip(mss.masses, masses):
#         vis.position = (mm.pos[0], mm.pos[1], mm.pos[2])

#     springs.geometry = LineSegmentsGeometry(positions=get_segments())
#     sleep(0.01)



#### ATTEMPT 2 

In [ ]:
import math
import sys
sys.path.append("../build/mechsystem")

from mass_spring import *

mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

def add_beam(c1, c2):
    p1 = mss[c1].pos
    p2 = mss[c2].pos
    L = math.dist(p1, p2)       
    mss.add(DistanceConstraint(c1, c2, L))

# 1) Rigid base footprint (3 FIX)
baseA = mss.add(Fix((0.0, -0.8, 0.0)))
baseB = mss.add(Fix((0.0,  0.8, 0.0)))
baseC = mss.add(Fix((0.8,  0.0, 0.0)))

# First tower node (mass) anchored to all 3
m0 = mss.add(Mass(3.0, (0.0, 0.0, 1.0)))
add_beam(baseA, m0)
add_beam(baseB, m0)
add_beam(baseC, m0)

# Also connect base triangle 
# add_beam(baseA, baseB)
# add_beam(baseB, baseC)
# add_beam(baseC, baseA)

# 2) Tower (vertical truss)
tower = [m0]
tower_height = 5
for z in range(2, tower_height + 1):
    mi = mss.add(Mass(2.5, (0.0, 0.0, float(z))))
    tower.append(mi)
    add_beam(tower[-2], tower[-1])   

# add diagonal bracing in tower 
for i in range(len(tower) - 1):
    if i + 1 < len(tower):
        pass

for i in range(len(tower) - 2):
    add_beam(tower[i], tower[i+2])

top = tower[-1]

# 3) Boom (horizontal truss)
boom = []
boom_len = 6
for i in range(1, boom_len + 1):
    mi = mss.add(Mass(1.5, (float(i), 0.0, float(tower_height))))
    boom.append(mi)
    if i == 1:
        add_beam(top, mi)
    else:
        add_beam(boom[-2], boom[-1])

# boom bracing
for i in range(len(boom) - 2):
    add_beam(boom[i], boom[i+2])

# stay cable / back brace (rigid in this model)
add_beam(tower[-2], boom[-1])

# 4) Load (hanging mass)
load = mss.add(Mass(4.0, (float(boom_len), 0.6, float(tower_height - 1.2))))
add_beam(boom[-1], load)

print("masses =", len(mss.masses), "joints =", len(mss.joints))


masses = 12 joints = 22


In [6]:
from pythreejs import *
from time import sleep

masses_vis = [
    Mesh(SphereBufferGeometry(0.18, 16, 16),
         MeshStandardMaterial(color="red"),
         position=(m.pos[0], m.pos[1], m.pos[2]))
    for m in mss.masses
]

fixes_vis = [
    Mesh(SphereBufferGeometry(0.25, 16, 16),
         MeshStandardMaterial(color="blue"),
         position=(f.pos[0], f.pos[1], f.pos[2]))
    for f in mss.fixes
]

def get_joint_segments():
    segs = []
    for j in mss.joints:
        pA = mss[j.connectors[0]].pos
        pB = mss[j.connectors[1]].pos
        segs.append([(pA[0],pA[1],pA[2]), (pB[0],pB[1],pB[2])])
    return segs

joints_vis = LineSegments2(
    LineSegmentsGeometry(positions=get_joint_segments()),
    LineMaterial(linewidth=4, color="yellow")
)

axes = AxesHelper(1)
view_width, view_height = 800, 450
camera = PerspectiveCamera(position=[10, 6, 8], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[
    *masses_vis, *fixes_vis, joints_vis,
    axes, camera, key_light, ambient_light
])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)
renderer


Renderer(camera=PerspectiveCamera(aspect=1.7777777777777777, position=(10.0, 6.0, 8.0), projectionMatrix=(1.0,…

In [7]:
from time import sleep
# for i in range(10000):
#     mss.simulate (0.02, 100)
#     for m,mvis in zip(mss.masses, masses):
#         mvis.position = (m.pos[0], m.pos[1], m.pos[2])

#     springpos = []
#     for s in mss.springs:
#         pA = mss[s.connectors[0]].pos
#         pB = mss[s.connectors[1]].pos
#         springpos.append ([ pA, pB ]) 
#     springs.geometry = LineSegmentsGeometry(positions=springpos)
#     sleep(0.01)

# for i in range(3000):
#     mss.simulate(0.003, 10)   # dt = 0.0003

#     for m, vis in zip(mss.masses, masses_vis):
#         vis.position = (m.pos[0], m.pos[1], m.pos[2])

#     joints_vis.geometry = LineSegmentsGeometry(positions=get_joint_segments())
#     sleep(0.01)




In [10]:
from time import sleep

for i in range(5000):
    mss.simulate (0.001, 100)  

    # update mass spheres
    for m, mvis in zip(mss.masses, masses_vis):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    # update beam segments from joints (distance constraints)
    jointpos = []
    for j in mss.joints:
        pA = mss[j.connectors[0]].pos
        pB = mss[j.connectors[1]].pos
        jointpos.append([(pA[0],pA[1],pA[2]), (pB[0],pB[1],pB[2])])

    joints_vis.geometry = LineSegmentsGeometry(positions=jointpos)

    sleep(0.01)


ValueError: Newton did not converge

### attempt 3 

In [ ]:
import math
import sys
sys.path.append("../build/mechsystem")
from mass_spring import *

mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

# --- helpers ---
def pos(c):
    p = mss[c].pos
    return (float(p[0]), float(p[1]), float(p[2]))

def dist(c1, c2):
    return math.dist(pos(c1), pos(c2))

def add_rigid(c1, c2):
    mss.add(DistanceConstraint(c1, c2, dist(c1, c2)))

def add_spring(c1, c2, k):
    mss.add(Spring(dist(c1, c2), float(k), (c1, c2)))


base = mss.add(Fix((0.0, 0.0, 0.0)))
m0 = mss.add(Mass(3.0, (0.0, 0.0, 1.0)))

add_rigid(base, m0)

guyA = mss.add(Fix((0.0, -0.8, 0.0)))
guyB = mss.add(Fix((0.0,  0.8, 0.0)))
add_spring(guyA, m0, k=2000.0)
add_spring(guyB, m0, k=2000.0)

tower = [m0]
tower_height = 5

for z in range(2, tower_height + 1):
    mi = mss.add(Mass(2.5, (0.0, 0.0, float(z))))
    tower.append(mi)
    add_rigid(tower[-2], tower[-1])

for i in range(len(tower) - 2):
    add_rigid(tower[i], tower[i + 2])

top = tower[-1]

boom = []
boom_len = 6

for i in range(1, boom_len + 1):
    mi = mss.add(Mass(1.5, (float(i), 0.0, float(tower_height))))
    boom.append(mi)
    if i == 1:
        add_rigid(top, mi)
    else:
        add_rigid(boom[-2], boom[-1])

for i in range(len(boom) - 2):
    add_rigid(boom[i], boom[i + 2])

add_spring(tower[-2], boom[-1], k=4000.0)


load = mss.add(Mass(4.0, (float(boom_len), 0.6, float(tower_height - 1.2))))
add_spring(boom[-1], load, k=1200.0)

print("masses =", len(mss.masses), "fixes =", len(mss.fixes),
      "joints =", len(mss.joints), "springs =", len(mss.springs))


masses = 12 fixes = 3 joints = 15 springs = 4


In [21]:
from pythreejs import *
from time import sleep

masses_vis = [
    Mesh(SphereBufferGeometry(0.18, 16, 16),
         MeshStandardMaterial(color="red"),
         position=(m.pos[0], m.pos[1], m.pos[2]))
    for m in mss.masses
]

fixes_vis = [
    Mesh(SphereBufferGeometry(0.25, 16, 16),
         MeshStandardMaterial(color="blue"),
         position=(f.pos[0], f.pos[1], f.pos[2]))
    for f in mss.fixes
]

def get_joint_segments():
    segs = []
    for j in mss.joints:
        pA = mss[j.connectors[0]].pos
        pB = mss[j.connectors[1]].pos
        segs.append([(pA[0],pA[1],pA[2]), (pB[0],pB[1],pB[2])])
    return segs

joints_vis = LineSegments2(
    LineSegmentsGeometry(positions=get_joint_segments()),
    LineMaterial(linewidth=4, color="yellow")
)

axes = AxesHelper(1)
view_width, view_height = 800, 450
camera = PerspectiveCamera(position=[10, 6, 8], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[
    *masses_vis, *fixes_vis, joints_vis,
    axes, camera, key_light, ambient_light
])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)
renderer


Renderer(camera=PerspectiveCamera(aspect=1.7777777777777777, position=(10.0, 6.0, 8.0), projectionMatrix=(1.0,…

In [22]:
from time import sleep

for i in range(1000):
    mss.simulate (0.01, 100)  

    # update mass spheres
    for m, mvis in zip(mss.masses, masses_vis):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    # update beam segments from joints (distance constraints)
    jointpos = []
    for j in mss.joints:
        pA = mss[j.connectors[0]].pos
        pB = mss[j.connectors[1]].pos
        jointpos.append([(pA[0],pA[1],pA[2]), (pB[0],pB[1],pB[2])])

    joints_vis.geometry = LineSegmentsGeometry(positions=jointpos)

    sleep(0.01)


ValueError: Newton did not converge

In [ ]:
from pythreejs import *
from time import sleep

mass_meshes = [
    Mesh(SphereBufferGeometry(0.12, 16, 16),
         MeshStandardMaterial(color="red"),
         position=(m.pos[0], m.pos[1], m.pos[2]))
    for m in mss.masses
]

fix_meshes = [
    Mesh(BoxBufferGeometry(0.18, 0.18, 0.18),
         MeshStandardMaterial(color="blue"),
         position=(f.pos[0], f.pos[1], f.pos[2]))
    for f in mss.fixes
]

def joint_segments():
    segs = []
    for j in mss.joints:
        pA = mss[j.connectors[0]].pos
        pB = mss[j.connectors[1]].pos
        segs.append([[float(pA[0]), float(pA[1]), float(pA[2])],
                     [float(pB[0]), float(pB[1]), float(pB[2])]])
    return segs

def spring_segments():
    segs = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        segs.append([[float(pA[0]), float(pA[1]), float(pA[2])],
                     [float(pB[0]), float(pB[1]), float(pB[2])]])
    return segs

j_geo = LineSegmentsGeometry(positions=joint_segments() or [[[0,0,0],[0,0,0]]])
j_mat = LineMaterial(linewidth=4, color="green")
j_line = LineSegments2(j_geo, j_mat)

s_geo = LineSegmentsGeometry(positions=spring_segments() or [[[0,0,0],[0,0,0]]])
s_mat = LineMaterial(linewidth=2, color="cyan")
s_line = LineSegments2(s_geo, s_mat)

axes = AxesHelper(1)

view_width, view_height = 900, 650
camera = PerspectiveCamera(position=[10, 8, 6], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight(intensity=0.6)

scene = Scene(children=[*mass_meshes, *fix_meshes, j_line, s_line, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)
renderer


Renderer(camera=PerspectiveCamera(aspect=1.3846153846153846, position=(10.0, 8.0, 6.0), projectionMatrix=(1.0,…

In [24]:
from pythreejs import LineSegmentsGeometry

for i in range(5000):
    mss.simulate(0.0005, 500)   

    for m, mesh in zip(mss.masses, mass_meshes):
        mesh.position = (float(m.pos[0]), float(m.pos[1]), float(m.pos[2]))

    j_segs = joint_segments()
    if j_segs:
        j_line.geometry = LineSegmentsGeometry(positions=j_segs)

    s_segs = spring_segments()
    if s_segs:
        s_line.geometry = LineSegmentsGeometry(positions=s_segs)

    sleep(0.01)


ValueError: Newton did not converge